In [ ]:
import toml
import re
import urllib
import pygsheets
import benchlingapi
import requests_html

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.cloning.workflow as workflow

# Code

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
session = benchlingapi.Session(config["benchling"]["api_key"])

# dnaplotlib

# Google Sheets

In [ ]:
PAULSSON_GDRIVE = "157G6iXC8pfXnI4HoiiRItWc5awDPNhmb"

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
api.google.get_drive_by_path(
    gc.drive.service, "Strains/LIB_Collection", root=PAULSSON_GDRIVE
)

In [ ]:
col = workflow.get_strain_collection_sheets(gc.drive.service, "LIB")
col

In [ ]:
workflow.get_next_collection_id(gc.open_by_key(col["strains"]).worksheet())

In [ ]:
workflow.get_next_collection_id(gc.open_by_key(col["oligos"]).worksheet())

In [ ]:
workflow.get_next_collection_id(gc.open_by_key(col["plasmids"]).worksheet())

In [ ]:
workflow.get_next_collection_id(
    gc.open_by_key(col["parts"]).worksheet_by_title("Parts")
)

In [ ]:
workflow.get_next_collection_id(gc.open_by_key(col["parts"]).worksheet_by_title("TUs"))

In [ ]:
workflow.get_next_collection_id(
    gc.open_by_key(col["parts"]).worksheet_by_title("Assemblies")
)

In [ ]:
def get_plasmid_maps():
    pass

In [ ]:
res = (
    gc.drive.service.files()
    .list(
        q=f"('{col['plasmid_maps']}' in parents) and (mimeType != 'application/vnd.google-apps.folder')"
    )
    .execute()
)

In [ ]:
res

# Benchling/Addgene

In [ ]:
folder = api.get_folder(session, "Test2", project="Test")

In [ ]:
dna = session.DNASequence.find_by_name("V37m", folder_id=folder.id)

In [ ]:
url = "https://www.addgene.org/108254/"
# url = 'https://www.addgene.org/108537/'
# url = 'http://www.addgene.org/121010/'
api.get_addgene_info(url)

In [ ]:
# url = 'https://www.addgene.org/kits/marionette-sensor-collection/#kit-contents'
url = "http://www.addgene.org/kits/murray-cidar-moclo-v1/"
kit = api.get_addgene_kit(url, include_info=False, include_sequences=False)

In [ ]:
# gb = get_genbank(addgene_sequences('http://www.addgene.org/121010/')['depositor_full'][0])[0]
gb = get_genbank(
    addgene_sequences("https://www.addgene.org/108537/")["addgene_full"][0]
)[0]

In [ ]:
dna = genbank_to_benchling(session, gb, "seq3", folder.id, accession="addgene_blah")
dna.save()